# Calculate 500-900hPa mean absolute vorticity

In [23]:
import os
import sys
sys.path.append('../..')
import glob
import time
import xarray
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean
import cimf as c
from ddxddy import ddxND, ddyND
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['font.cursive'] = ['Helvetica-Oblique']
plt.rcParams['pdf.fonttype'] = 42

In [3]:
p_ref = 100000
cp = 1004
Rdry = 286.9
kappa = Rdry/cp
fc = 4.25e-5

In [4]:
dataloc = "/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989interped/"
dataloc2 = "/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989/"
files = sorted(glob.glob(dataloc+'*.nc'))
files2 = sorted(glob.glob(dataloc2+'*.nc'))

In [5]:
from numba import float64, guvectorize

@guvectorize(
    "(float32[:], float32[:], float32[:], float32[:])",
    " (n), (n), (m) -> (m)",
    nopython=True,
)
def interp1d_gu(f, x, xi, out):
    """Interpolate field f(x) to xi in ln(x) coordinates."""
    i, imax, x0, f0 = 0, len(xi), x[0], f[0]
    while xi[i]<x0 and i < imax:
        out[i] = np.nan      
        i = i + 1 
    for x1,f1 in zip(x[1:], f[1:]):
        while xi[i] <= x1 and i < imax:
            out[i] = (f1-f0)/np.log(x1/x0)*np.log(xi[i]/x0)+f0
            i = i + 1
        x0, f0 = x1, f1
    while i < imax:
        out[i] = np.nan
        i = i + 1


def xr_interp(data, p, newp):

    interped = xarray.apply_ufunc(
        interp1d_gu,  # first the function
        data,  # now arguments in the order expected by 'interp1_np'
        p,  # as above
        newp,  # as above
        input_core_dims=[['hybrid'], ['hybrid'], ['p']],  # list with one entry per arg
        output_core_dims=[['p']],  # returned data has one dimension
        exclude_dims=set(('hybrid',)),  # dimensions allowed to change size. Must be a set!
        dask="parallelized",
        output_dtypes=[
            data.dtype
        ],  # one per output; could also be float or np.dtype("float64")
    ).compute()
    interped['p'] = newp

    return interped

In [ ]:
%%time
ds2 = xarray.open_mfdataset('/Users/jasperdejong/Documents/PhD/Irma/Data/LambertGrid/629x989/fc2017090512+???.nc',
                          combine='nested', concat_dim='valid_time')
ds2['pres'] = (ds2.a + ds2.b * ds2.p0m).astype('float32')
ds2['eta'] = ddxND(ds2.v) - ddyND(ds2.u)
ds2['theta'] = (ds2.t  * (p_ref/ds2.pres)**kappa).astype('float32')
ds2 = c.crop(ds2, d=80)
ds2 = c.convert_wind(ds2)
ds2['u_rad'] = ds2.u_rad.astype('float32')
ds2['v_tan'] = ds2.v_tan.astype('float32')
pi = np.arange(25,1025,25,dtype=ds2.pres.dtype)*100
ds2['eta'] = xr_interp(ds2.eta, ds2.pres, pi)
ds2['theta'] = xr_interp(ds2.theta, ds2.pres, pi)
ds2['u_rad'] = xr_interp(ds2.u_rad, ds2.pres, pi)
ds2['v_tan'] = xr_interp(ds2.v_tan, ds2.pres, pi)
r, angle = c.distance(ds2.latitude.sel(dy=0,dx=0), ds2.longitude.sel(dy=0,dx=0), 
                      ds2.latitude, ds2.longitude)
ds2 = ds2.assign_coords({'r':r})

In [18]:
data = ds2[[var for var in ds2.variables if ('hybrid' not in ds2[var].dims)]]

In [22]:
data.to_netcdf('fig/eta_500-900hPa-mean_hovmoller/figdata2.nc')